In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import tensorflow_hub as hub

In [3]:
train_data, validation_data= tfds.load(
    name="imdb_reviews",
    split=[ 'train','test'],
    as_supervised=True)

In [4]:
train_data.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [5]:
embedding = "nnlm-en-dim50_2/"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [6]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 64)                3264      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 48,198,089
Trainable params: 48,198,089
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [9]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=4,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/4
49/49 [==============================] - 29s 598ms/step - loss: 0.0032 - accuracy: 0.9997 - val_loss: 0.8119 - val_accuracy: 0.8474
Epoch 2/4
49/49 [==============================] - 29s 596ms/step - loss: 0.0021 - accuracy: 0.9998 - val_loss: 0.8597 - val_accuracy: 0.8468
Epoch 3/4
49/49 [==============================] - 30s 603ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.9010 - val_accuracy: 0.8468
Epoch 4/4
49/49 [==============================] - 30s 603ms/step - loss: 9.3217e-04 - accuracy: 1.0000 - val_loss: 0.9370 - val_accuracy: 0.8465


In [10]:
model.save('model_py/model.h5')